In [ ]:
"""
1. 패션데이터셋 읽어들이기 : 변수는 기존과 동일
2. 데이터 스케일링 : 변수는 기존과 동일
3. 2차원 데이터로 변경
4. 훈련 : 검증 = 8:2로 분리 : 변수는 기존과 동일
"""

In [1]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "0"

In [ ]:
### 텐서플로우 프레임워크(패키지라고도 칭함)
import tensorflow as tf

### 케라스 라이브러리 불러들이기
from tensorflow import keras

### 시각화 라이브러리 불러들이기
import matplotlib.pyplot as plt

### 넘파이
import numpy as np

### 훈련 및 테스트 데이터로 분류하는 라이브러리
from sklearn.model_selection import train_test_split

### 딥러닝 랜덤 규칙 정의하기
# - 딥러닝에서의 랜덤 규칙은 항상 일정하게 유지되지는 않음
# - 딥러닝 모델 내부에서 훈련을 위한 데이터를 임의로 추출하여 사용(사람이 관여 안함)
tf.keras.utils.set_random_seed(42)

# 🚫 아래 줄은 GPU에서 오류 나는 연산을 강제하므로 주석 유지 또는 완전히 제거
# tf.config.experimental.enable_op_determinism()


In [3]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
train_scaled_255 = train_input / 255.0
test_scaled_255  = test_input / 255.0

print(train_scaled_255.shape, train_target.shape)
print(test_scaled_255.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [5]:
train_scaled_2d = train_scaled_255.reshape(-1, 28 * 28)
test_scaled_2d = test_scaled_255.reshape(-1, 28 * 28)

print(train_scaled_2d.shape, train_target.shape)
print(test_scaled_2d.shape, test_target.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [6]:
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled_2d, train_target, test_size=0.2, random_state=42)

### 훈련 (train_scaled, train_target)
print(train_scaled.shape, train_target.shape)
### 검증 (val_scaled, val_target)
print(val_scaled.shape, val_target.shape)

(48000, 784) (48000,)
(12000, 784) (12000,)


### 신경망 모델에 계층(Layer) 추가하는 방법(3가지)

##### 1. 계층(Layer)을 먼저 만들고, 신경망 모델 생성 시 추가하는 방법

In [7]:
### 입력계층 생성하기
# - 변수명 : dense1
# - 활성화 함수 : sigmoid 사용
# - 출력 데이터 갯수 : 100개
# - 입력 데이터 갯수 : 784개
dense1 = keras.layers.Dense(
    # 출력갯수
    units = 100,
    # 활성화 함수
    activation = "sigmoid",
    # 입력 데이터 갯수
    input_shape = (784, )
)
dense1

In [8]:
### 출력계층 생성하기
dense2 = keras.layers.Dense(
    units = 10,
    activation = "softmax",
)
dense2

In [9]:
### 신경망 모델 생성과 동시에 미리 생성한 계층 추가하기
model = keras.Sequential([dense1, dense2])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


##### 2. 신경망 모델 생성 시에 계층(Layer)을 함께 추가하는 방식

In [10]:
### 신경망 모델 생성하기
model2 = keras.Sequential([
    # 입력계층 생성하기
    keras.layers.Dense(units=100, activation="sigmoid", 
                       input_shape=(784, ), name="Input_Layer"),
    # 출력계층 생성하기
    keras.layers.Dense(units=10, activation="softmax", 
                       name="Output_Layer"),
    # 모델 이름 정의
], "Model2")

model2.summary()

Model: "Model2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 100)               78500     
                                                                 
 Output_Layer (Dense)        (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


##### 3. 신경망 모델을 먼저 생성 후 add()함수를 이용해서 계층(Layer) 추가하는 방식(가장 많이 사용됨)

In [11]:
model3 = keras.Sequential()
model3

In [12]:
### 입력계층 생성 및 모델에 추가하기
model3.add(
    keras.layers.Dense(units=100, activation="sigmoid",
                       input_shape=(784, ), name="Input-Layer")
)

In [13]:
### 출력계층 생성 및 모델에 추가하기
model3.add(
    keras.layers.Dense(units=10, activation="softmax",
                       name="Output-Layer")
)

In [14]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input-Layer (Dense)         (None, 100)               78500     
                                                                 
 Output-Layer (Dense)        (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
"""
1. 모델 환경 설정하기
2. 훈련 데이터로 10회만 훈련
"""

In [15]:
model3.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model3.fit(train_scaled, train_target, epochs=50)

Epoch 1/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5652 - accuracy: 0.8081
Epoch 2/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.4096 - accuracy: 0.8521
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3743 - accuracy: 0.8658
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3507 - accuracy: 0.8735
Epoch 5/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3327 - accuracy: 0.8784
Epoch 6/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3213 - accuracy: 0.8845
Epoch 7/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3093 - accuracy: 0.8877
Epoch 8/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2995 - accuracy: 0.8906
Epoch 9/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2892 - accuracy: 0.8946
Epoch 10/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.280

In [16]:
### 훈련데이터로 평가하기
train_score = model3.evaluate(train_scaled, train_target)
print(f"훈련 손실율 : {train_score[0]}, 훈련 정확도 : {train_score[1]}")

### 검증데이터로 평가하기
val_score = model3.evaluate(val_scaled, val_target)
print(f"검증 손실율 : {val_score[0]}, 검증 정확도 : {val_score[1]}")

1500/1500 [==============================] - 2s 1ms/step - loss: 0.1417 - accuracy: 0.9500
훈련 손실율 : 0.1416841596364975, 훈련 정확도 : 0.949999988079071
375/375 [==============================] - 0s 1ms/step - loss: 0.4032 - accuracy: 0.8845
검증 손실율 : 0.403218150138855, 검증 정확도 : 0.8845000267028809


In [ ]:
"""
<모델 성능 향상 방법>
 - 데이터 증가시키기
 - 하이퍼파라미터 튜닝
  -- 반복횟수 증/감
  -- 출력계층을 제외한 계층에서의 활성화(activation)함수 변경
  -- 배치사이즈
  -- 출력계층을 제외한 계층에서의 출력 데이터의 갯수(units)
  -- 은닉계층(Hidden Layer)을 추가 또는 제거(일반적으로 추가 후 영향이 없으면 제거)
"""

### 성능향상 - 은닉계층(Hidden Layer)추가

In [ ]:
"""
1. 모델생성 : 변수명은 model
2. 출력계층 : 기존과 동일
3. 은닉계층 : 자유롭게
4. 출력계층
5. 훈련 횟수 10회로 훈련까지 진행
"""

In [27]:

model = keras.Sequential([
    keras.Input(shape=(784,)),
    keras.layers.Dense(units=100, activation="sigmoid", name="Input_Layer"),
    keras.layers.Dense(units=50, activation="relu", name="Hidden_Layer"),
    keras.layers.Dense(units=10, activation="softmax", name="Output_Layer")
])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_Layer (Dense)         (None, 100)               78500     
                                                                 
 Hidden_Layer (Dense)        (None, 50)                5050      
                                                                 
 Output_Layer (Dense)        (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.fit(train_scaled, train_target, epochs=10, batch_size=32)

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3124 - accuracy: 0.8842
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3013 - accuracy: 0.8879
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2898 - accuracy: 0.8925
Epoch 4/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2797 - accuracy: 0.8966
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2704 - accuracy: 0.8996
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2649 - accuracy: 0.9030
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2566 - accuracy: 0.9053
Epoch 8/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2502 - accuracy: 0.9072
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2436 - accuracy: 0.9099
Epoch 10/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.237

In [31]:
train_score = model.evaluate(train_scaled, train_target)
print(f"훈련 손실율 : {train_score[0]}, 훈련 정확도 : {train_score[1]}")

val_score = model.evaluate(val_scaled, val_target)
print(f"검증 손실율 : {val_score[0]}, 검증 정확도 : {val_score[1]}")

1500/1500 [==============================] - 2s 1ms/step - loss: 0.2232 - accuracy: 0.9188
훈련 손실율 : 0.22321775555610657, 훈련 정확도 : 0.918833315372467
375/375 [==============================] - 0s 1ms/step - loss: 0.3447 - accuracy: 0.8850
검증 손실율 : 0.3447072207927704, 검증 정확도 : 0.8849999904632568


##### 성능향상 : 모델 설정 시 옵티마이저(Optimizer) 설정(최적화 기법)

In [ ]:
"""
<옵티마이저(Optimizer, 최적화) 기법>
 - 손실을 줄여나가기 위한 최적화(Optimizer) 방법을 의미함
 - 손실을 줄여나가는 최적화 방법으로 "경사하강법" 이론이 적용됩니다.
 - 특성들의 시작 위치에서 목적지(종속변수 위치)까지 도달하기 위한
   기울어진 방향을 찾기 위한 방법을 의미합니다.
 - "경사하강법" 이론을 적용한 여러가지 방법들중 하나를 선택하여 정의
 - 옵티마이저(최적화) 방법
   : SGD(확률적경사하강법) < Adagrad < RMSProp < Adam, 이외 등등
 - 옵티마이저 설정 위치 : model.compile(optimizer="옵티마이저(최적화) 방법중 1개")
 
<옵티마이저(최적화) 방법 정의>
 * SGD(확률적 경사하강법)
  - 특성이 현재 위치에서 목적지까지 도달하는 과정 중에 보폭을 크게하여
    많은 길을 거치면서(극단적으로 방향을 바꿉니다.) 빠르게 탐색
  - 지그 재그 모양으로 탐색하면서 나아가는 방법
  - 아래 옵티마이저 방법들은 SGD를 근간으로 향상된 방법들 입니다.
  - 단점 : 보폭을 크게하면서 방향을 근단적으로 바꾸기 때문에
           주변을 정밀하게 확인하기 어려움
         : 보폭을 크게하기 때문에, 목적지를 건너 띄는 경우도 발생함
                                 (종속변수를 잘 못 찾는 경우 발생)
   
 * Adagrad
  - SGD의 큰 보폭에 대한 단점을 보완한 방법...
  - 학습률(보폭)을 적절하게 설정하기 위해 학습률 감소(보폭을 짧게)라는 기술 사용
  - 학습 진행 중에 학습률을 줄여가는 방법 사용
  - 처음에는 학습률(보폭)을 크게 학습하다가,
    점점 작게(보폭을 짧게) 학습한다는 개념을 적용
  - 이미 학습된 곳은 보폭을 크게하고, 학습이 완료되었던 곳은 보폭을 짧게 세밀하게 탐색
  - 손실이 더 이상 줄어들지 않으면(손실이 0이면) 종료
  - 단점, 목적지에 도달하지 않더라도 손실이 0이면 종료하는 단점이 있음
 
 * RMSProp
  - Adagrad는 학습량을 점점 작게 학습하기 때문에
    학습률(보폭)이 0이 되어 갱신되지 않는(학습되지 않는) 시점이 발생할 수 있는
    단점이 있음.
  - Adagrad의 단점을 보완하여,
    과거(이전)의 기울기 값을 반영하는 방식을 적용함
  - 먼 과거의 기울기(경사) 값은 조금만 반영하고,
    최근 기울기(경사)를 많이 반영하는 방식으로 처리됨
  - 과거 데이터를 저장해 놓아야 하기 때문에, 다소 훈련 시간이 걸림
  - 옵티마이저의 기본값(default)으로 사용됨, 생략가능
 
 * Adam
  - 공이 굴러가듯이 모멘텀(Momentum, 관성=방향담당)과 RMSProp을 융합한 방법
  - 방향(momentum)과 학습률(보폭)을 적절하게하여 탐색함
  - 자주 사용되는 기법으로, 좋은 결과를  얻을 수 있는 방법으로 유명함
 
 * Momentum(모멘텀)
  - 관성과 가속도를 적용하여 이동하던 방향으로 좀 더 유연하게 작동함
  - 메모리 사용이 많은 단점이 있음
    (과거 데이터를 저장해 놓고, 다음 과정에서 방향성(관성)을 이어 받아서 사용하게 됨)
"""



In [34]:
model.compile(
    optimizer = "sgd",
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.fit(train_scaled, train_target, epochs=10, batch_size=32)

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1985 - accuracy: 0.9280
Epoch 2/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1915 - accuracy: 0.9302
Epoch 3/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1895 - accuracy: 0.9304
Epoch 4/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1879 - accuracy: 0.9314
Epoch 5/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1864 - accuracy: 0.9320
Epoch 6/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1858 - accuracy: 0.9321
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1845 - accuracy: 0.9323
Epoch 8/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.1838 - accuracy: 0.9330
Epoch 9/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1829 - accuracy: 0.9338
Epoch 10/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.182

In [35]:
model.compile(
    optimizer = "adam",
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.fit(train_scaled, train_target, epochs=10, batch_size=32)

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2250 - accuracy: 0.9179
Epoch 2/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2160 - accuracy: 0.9192
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2114 - accuracy: 0.9203
Epoch 4/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2049 - accuracy: 0.9224
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1987 - accuracy: 0.9251
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1931 - accuracy: 0.9261
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1861 - accuracy: 0.9304
Epoch 8/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1828 - accuracy: 0.9304
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1761 - accuracy: 0.9344
Epoch 10/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.172

In [41]:
model.compile(
    optimizer = "adagrad",
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.fit(train_scaled, train_target, epochs=10, batch_size=64)

Epoch 1/10
750/750 [==============================] - 2s 2ms/step - loss: 0.1262 - accuracy: 0.9543
Epoch 2/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1259 - accuracy: 0.9542
Epoch 3/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1257 - accuracy: 0.9544
Epoch 4/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1255 - accuracy: 0.9543
Epoch 5/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1252 - accuracy: 0.9543
Epoch 6/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1251 - accuracy: 0.9546
Epoch 7/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1249 - accuracy: 0.9545
Epoch 8/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1247 - accuracy: 0.9547
Epoch 9/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1246 - accuracy: 0.9546
Epoch 10/10
750/750 [==============================] - 1s 2ms/step - loss: 0.1245 - accuracy: 0.9549

##### 성능향상 : 옵티마이저에 학습률(보폭의 크기) 정의하기

In [ ]:
"""
<학습률(Learning Rate)>
 - 경사를 내려 올 때의 "보폭"이라고 이해
 - 옵티마이저 4개는 객체(클래스)로 되어 있습니다.
 - 클래스 생성 시에 학습률(learning_rate)을 정의 할 수 있습니다.
 - 학습률이 작을  수록 -> 보폭이 작음
           높을 수록  -> 보폭이 큼
 - 가장 손실이 작은 위치를 찾아서 움직이도록 작동됩니다.
 - 이때 가장 손실이 작은 위치는 모델이 스스로 찾아주기에 사람이 관여하지 않습니다.
 - 학습률(learning_rate) 값의 범위 : 0.1~0.0001 (기본값은 0.01, 생략가능)
 - 학습률의 값에 따라서 -> 훈련 성능에 영향을 미치기에
                           => 하이퍼파라메터 대상이 됩니다.


 - 과적합을 해소하기 위한 튜닝 방법으로 사용되기도 합니다.
   * 과대적합이 발생한 경우 : 학습률을 크게(보폭을 크게) 합니다.
   * 과소적합이 발생한 경우 : 학습률을 작게(보폭을 작게) 합니다.
"""

In [46]:
### 옵티마이저 클래스 생성하기 : SGD 사용
sgd = keras.optimizers.SGD(learning_rate = 0.1)

### 모델 설정하기
model.compile(
    optimizer = sgd,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.fit(train_scaled, train_target, epochs=1000, batch_size=512)

print("\n----------------------------------------------------------------------\n")

train_score = model.evaluate(train_scaled, train_target)
print(f"훈련 손실율 : {train_score[0]}, 훈련 정확도 : {train_score[1]}")

print("\n----------------------------------------------------------------------\n")

val_score = model.evaluate(val_scaled, val_target)
print(f"검증 손실율 : {val_score[0]}, 검증 정확도 : {val_score[1]}")

Epoch 1/1000
94/94 [==============================] - 1s 4ms/step - loss: 0.1010 - accuracy: 0.9641
Epoch 2/1000
94/94 [==============================] - 0s 3ms/step - loss: 0.0973 - accuracy: 0.9657
Epoch 3/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0966 - accuracy: 0.9653
Epoch 4/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0961 - accuracy: 0.9661
Epoch 5/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0956 - accuracy: 0.9657
Epoch 6/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0950 - accuracy: 0.9662
Epoch 7/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0950 - accuracy: 0.9662
Epoch 8/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0947 - accuracy: 0.9665
Epoch 9/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0946 - accuracy: 0.9664
Epoch 10/1000
94/94 [==============================] - 0s 2ms/step - loss: 0.0942 - accuracy: 0.9666

In [ ]:
"""
<딥러닝에서 사용하는 숫자들>
 512 > 256 > 128 > 64 > 32 > 16 > 8
 
  - 배치사이즈를 계층마다 사용시에는 입력 받는 데이터의 갯수를 감안하여 배치사이즈 정의
  - 배치사이즈 갯수는 보통 512 > 256 > *128 > 64 > 32 > 16 > 8 순으로...
"""

In [ ]:
"""
<모멘텀(momentum)>
 - 과거에 학습된 방향(기울기)를 기억하고 있다가,
   다음의 학습 방향을 찾을 때 -> 관성을 적용시키는 방법
 - 기본적으로 0.9 이상의 값을 사용합니다.
   (기본값 = 0)
 - 보통 nesterov=True 속성과 함께 사용됩니다.
   * nesterov : 모멘텀 방향보다 조금더 앞서서 경사를 계산해 놓는 방식(미리 체크)
              : 미리 알고 있어야 방향에 관성을 적용하여 명확하게 탐색할 수 있음
 - 모멘텀 속성을 사용할 수 있는 옵티마이저 : 주로 SGD 및 Adam
"""


In [48]:
### SGB 옵티마이저에 학습율, 모멘텀 추가하기
sgd = keras.optimizers.SGD(
    # 모멘텀 적용
    momentum = 0.9,
    
    # 미리 앞서서 경사확인
    nesterov = True,
    
    # 학습률(보폭)
    learning_rate = 0.1
)

### 모델 환경 설정에 적용
model.compile(
    optimizer = sgd,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

### 훈련시키기
model.fit(train_scaled, train_target, epochs=50, batch_size=128)

Epoch 1/50
375/375 [==============================] - 1s 2ms/step - loss: 0.3867 - accuracy: 0.8779
Epoch 2/50
375/375 [==============================] - 1s 2ms/step - loss: 0.2410 - accuracy: 0.9105
Epoch 3/50
375/375 [==============================] - 1s 2ms/step - loss: 0.2173 - accuracy: 0.9176
Epoch 4/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1975 - accuracy: 0.9255
Epoch 5/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1879 - accuracy: 0.9271
Epoch 6/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1803 - accuracy: 0.9306
Epoch 7/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1736 - accuracy: 0.9345
Epoch 8/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1689 - accuracy: 0.9355
Epoch 9/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1624 - accuracy: 0.9376
Epoch 10/50
375/375 [==============================] - 1s 2ms/step - loss: 0.1554 - accuracy: 0.9399

In [49]:
### 성능 검증(평가)
train_score = model.evaluate(train_scaled, train_target)
print(f"훈련 손실율 : {train_score[0]}, 훈련 정확도 : {train_score[1]}")

print("\n----------------------------------------------------------------------\n")

val_score = model.evaluate(val_scaled, val_target)
print(f"검증 손실율 : {val_score[0]}, 검증 정확도 : {val_score[1]}")

1500/1500 [==============================] - 2s 1ms/step - loss: 0.0735 - accuracy: 0.9726
훈련 손실율 : 0.07347985357046127, 훈련 정확도 : 0.9725624918937683

----------------------------------------------------------------------

375/375 [==============================] - 0s 1ms/step - loss: 0.5849 - accuracy: 0.8865
검증 손실율 : 0.5848613977432251, 검증 정확도 : 0.8865000009536743
